In [1]:
# Dependencies
%matplotlib osx
import stf
import eventanalysis
import utilities
import matplotlib.pyplot as plt
import minievents
import numpy as np
from scipy import stats, interpolate, optimize, integrate
import json

/Users/douglasboubert/anaconda3/envs/py27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Reading in file


In [128]:
# Reload when edits happen
reload(utilities)
reload(stf)
reload(eventanalysis)

Reading in file


<module 'eventanalysis' from 'eventanalysis.pyc'>

In [3]:
# Analyse big events and remove noise
reload(eventanalysis)
EA = eventanalysis.EventAnalysis()
EA.run()

Identified events
Merged events
Quantified events
Showed big events
Obtained Fourier amplitudes
Assessed significance of Fourier amplitudes
Grouped spikes together
Processed spike groups
Removed spikes


In [130]:
%%time
# Analyse minievents
reload(utilities)
reload(minievents)
MEH = minievents.MiniEventHandler(EA.data[-1],template_name='triexponential',mask=EA.mask,event_number_barrier=10000,bayes_bound=0.0,score_bound=1.5)
MEH.run()

Classified background noise
8.90336489677
Completed initial classification of events
1 34374 6840.444517315589
2 72360 14399.67898041997
3 74276 14780.9640125715
4 55977 11139.453154877952
5 54732 10891.69748419494
6 60794 12098.03874980171
7 36148 7193.471472971546
8 9646 1919.5591963119268
9 92527 18412.92284440739
10 161694 32177.193104754377
11 149977 29845.50379279223
12 59507 11841.925056493244
13 157844 31411.041030754695
14 79981 15916.262085861936
15 89105 17731.943000971827
16 167685 33369.40533211336
17 138002 27462.472341845172
18 13247 2636.160136175005
19 94105 18726.945694477905
20 52165 10380.86310134892
21 41111 8181.11114654568
22 106524 21198.333384608304
23 75132 14951.30847369974
24 52601 10467.62733622265
25 151239 30096.642472633164
26 10388 2067.217596028229
27 52343 10416.285197237736
28 20680 4115.331140341142
29 71212 14171.226361990974
30 150177 29885.303900532475
31 53570 10660.458858224127
32 97203 19343.44936337427
33 145685 28991.393480686613
34 32128 63

286 113208 22528.45298528723
287 17716 3525.493543630738
288 123578 24592.08857161884
289 145401 28934.877327695467
290 102963 20489.692466293276
291 27217 5416.197661830988
292 33727 6711.691168775903
293 53422 10631.006778496347
294 35159 6996.659940196044
295 102435 20384.620181859034
296 119521 23784.743386108006
297 82679 16453.165539277816
298 20262 4032.148915164033
299 156577 31158.907348220255
300 9487 1887.9181106584335
301 96325 19168.726890394606
302 4687 932.7155248925981
303 105986 21091.27109478705
304 88639 17639.20874993706
305 110870 22063.189725803786
306 147467 29346.01244065218
307 74037 14733.40288382191
308 29172 5805.243714991865
309 50552 10059.875232423858
310 120488 23977.17690703208
311 95308 18966.343342535467
312 32396 6446.821451764585
313 42273 8412.349772516493
314 3373 671.2288170392006
315 151131 30075.150414453434
316 92259 18359.590700035464
317 149371 29724.909466339293
318 79447 15809.995798195487
319 166472 33128.01767866879
320 121325 24143.7403

572 77996 15521.246016540023
573 156316 31106.968207619237
574 110790 22047.26968270769
575 115524 22989.338232919246
576 40250 8009.771682723933
577 99094 19719.75938205827
578 54877 10920.552562306615
579 58353 11612.278434832042
580 65532 13040.90330216807
581 28291 5629.924240396094
582 85316 16977.92995983292
583 91169 18142.680112851136
584 156785 31200.299460270107
585 5498 1094.1049617792842
586 23649 4706.16373974505
587 107998 21491.660178653896
588 64489 12833.3457403027
589 85973 17108.67331375962
590 154753 30795.930365629236
591 54918 10928.711584393366
592 14445 2874.5627815390612
593 83190 16554.854814554135
594 59246 11789.985915892226
595 135102 26885.370779611647
596 28502 5671.9133540620505
597 16524 3288.2849014988888
598 81332 16185.111813647278
599 26688 5310.926376858045
600 96178 19139.473811205527
601 76224 15168.617061961468
602 134237 26713.235313635098
603 111231 22135.028920274926
604 86062 17126.38436170403
605 76536 15230.705230036247
606 8235 1638.76943

In [131]:
MEH._post_process_event_box(significance_threshold=2.0,charge_threshold=5.0,rchi2_threshold=1.0)
plt.plot(MEH.t,MEH.data)
plt.plot(MEH.t,MEH.data_residual)
plt.plot(MEH.t,MEH.score)
plt.scatter(MEH.raw_event_box['t'],np.zeros(MEH.raw_event_box['t'].size),zorder=10,c='purple')
plt.scatter(MEH.event_box['t'],np.zeros(MEH.event_box['t'].size),zorder=10)
#plt.xlim([17355,17365])
#plt.ylim([-20,20])

In [45]:
# 19758
#
T = np.linspace(0,100,1000)
plt.plot(T,utilities._triexponential(T,MEH.raw_event_box['params'][0]))

In [124]:
MEH.event_box['f']

array([0.999     , 0.99897971, 0.999     , 0.95277877, 0.97299746,
       0.98845852, 0.999     , 0.001     , 0.00671531, 0.66704547,
       0.84224791])

In [132]:
plt.loglog(MEH.event_box['rise'],MEH.event_box['decay'],ls='none',marker='o')

In [133]:
from sklearn.cluster import KMeans
X = np.stack([np.log10(MEH.event_box['rise']),np.log10(MEH.event_box['decay'])]).T
n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(X)
kmeans.labels_
for i in range(n_clusters):
    inclust = np.where(kmeans.labels_==i)
    plt.loglog(MEH.event_box['rise'][inclust],MEH.event_box['decay'][inclust],ls='none',marker='o')

In [ ]:
# Plot all the events together
e_time = np.linspace(0.,10.,1000)
colorlist= ['deepskyblue','orangered','forestgreen','crimson']
for i in range(MEH.event_box['N']):
    plt.plot(e_time,(MEH.event_box['scale'][i])*MEH._biexponential(e_time,RISE=MEH.event_box['rise'][i],DECAY=MEH.event_box['decay'][i]),c=colorlist[kmeans.labels_[i]],alpha=0.2)

In [ ]:
plt.loglog(MEH.event_box['peak_time'],MEH.event_box['rise'],ls='none',marker='o')

In [ ]:
for i in range(MEH.event_box['rchi2'].size):
    if MEH.event_box['rchi2'][i]>1.2:
        print MEH.event_box['rchi2'][i], int(MEH.event_box['t'][i]/MEH.dt), MEH.event_box['t'][i]

In [ ]:
bins = np.arange(-1,10,0.2)
plt.hist(MEH.event_box['bayes_factor'],bins)

In [ ]:
#MEH.event_box['rchi2'][340]
MEH.data.size

In [ ]:
bins= np.arange(0,100,1)
plt.hexbin(MEH.event_box['area'],-MEH.event_box['scale']/MEH.event_box['noise'],mincnt=1)

In [ ]:
MEH._post_process_event_box()
plt.plot(MEH.t,MEH.data)
plt.plot(MEH.t,MEH.data_residual)
plt.plot(MEH.t,MEH.score)
plt.scatter(MEH.raw_event_box['t'],np.zeros(MEH.raw_event_box['t'].size),zorder=10,c='purple')
colorlist= ['deepskyblue','orangered','forestgreen','crimson']
for ic in range(n_clusters):
    inclust = np.where(kmeans.labels_==ic)
    plt.scatter(MEH.event_box['t'][inclust],np.zeros(MEH.event_box['t'][inclust].size),zorder=10,c=colorlist[ic])

In [ ]:
_score = -np.fft.irfft(MEH.ft_data * MEH.ft_template.conjugate(),n=MEH.data.size)
#_SCORE = (_score-MEH.score_med)/MEH.score_std
_SCORE = (_score-left_score_med)/left_score_std
sc_bins = np.arange(start=_SCORE.min()-1.0,stop=_SCORE.max()+1.0,step=0.1)
sc_midbins = (sc_bins[1:]+sc_bins[:-1])/2.0
lnkde = stats.gaussian_kde(_SCORE).logpdf(sc_midbins)
sc_lnkde_interp = interpolate.interp1d(sc_midbins,lnkde)
sc_lnev = 2.0*(sc_lnkde_interp(_SCORE)-stats.norm.logpdf(_SCORE)+MEH.log_bayes_weight)

In [ ]:
bins = np.arange(-10,10,0.01)
plt.hist(sc_lnev,bins)
plt.yscale('log')

In [ ]:
from matplotlib.colors import LogNorm
plt.hexbin(sc_lnev,_SCORE,mincnt=1,norm=LogNorm(),gridsize=200,extent=[-10,10,-5,5])

In [ ]:
int(18413/MEH.dt)

In [ ]:
valid = np.where((_SCORE>1.5))
plt.plot(MEH.t,MEH.data)
plt.scatter(MEH.t[valid],np.zeros(valid[0].size),zorder=10,c='orangered')
plt.plot(MEH.t,_SCORE)
plt.plot(MEH.t,_score)

In [ ]:
plt.plot(sc_midbins,lnkde)
plt.plot(sc_midbins,stats.norm.logpdf(sc_midbins,scale=1.0))

In [ ]:
plt.hist(_SCORE,100)
plt.yscale('log')

In [ ]:
all_score_med, all_score_std = utilities.rolling(_score,FUNC='SUFFICIENT_STATISTICS',WINDOW=int(0.1*MEH.median_window))
left_score_med, left_score_std = utilities.rolling(_score,FUNC='SUFFICIENT_STATISTICS_LEFT',WINDOW=int(0.1*MEH.median_window))

In [ ]:
_LEFT_SCORE = (_score-left_score_med)/left_score_std
_ALL_SCORE = (_score-all_score_med)/all_score_std

In [ ]:
bins = np.linspace(-10,20,100)
plt.hist(_LEFT_SCORE,bins,alpha=0.5)
plt.hist(_ALL_SCORE,bins,alpha=0.5)
plt.yscale('log')

In [115]:
reload(utilities)
template_name = 'triexponential'
_template, _template_peak, _template_area, _template_params_names, _template_params_ranges, _template_params_defaults = utilities.obtain_template(TEMPLATE_NAME=template_name)
from scipy import signal
_mean_start_index = 34374
_n_points = int(10.0/MEH.dt)

NOISE = MEH.noise_std[_mean_start_index]
MEDIAN = MEH.noise_med[_mean_start_index]
_peaks_bool = True
_peaks = [_n_points]
_previous_peaks = []
_first_peak = _mean_start_index+min(_peaks)-_n_points
_last_peak = _mean_start_index+max(_peaks)-_n_points
_default_peak_time = _template_peak(PARAMS=MEH._template_params_defaults())
print _default_peak_time
_left_extension = _n_points
_right_extension = int(2.0*_n_points)

while _peaks_bool:
    _score = MEH.score_initial[_first_peak-_left_extension:_last_peak+_right_extension]
    _data = MEH.data[_first_peak-_left_extension:_last_peak+_right_extension]
    _time = MEH.t[_first_peak-_left_extension:_last_peak+_right_extension]
    _peaks = []
    _peaks_heights = []
    _suggested_peaks, _peaks_props  = signal.find_peaks(-_data,prominence=NOISE*2,height = MEH.min_peak_current+NOISE,width=2)
    if len(_suggested_peaks) == 0:
        MEH.score[-10+_mean_start_index:10+_mean_start_index] = 0.0
    for _peak_idx,_peak_height in zip(_suggested_peaks,_peaks_props['peak_heights']):
        _mad_idx = int((_time[_peak_idx]-_default_peak_time)/MEH.dt)
        _max_score = MEH.score_initial[_mad_idx]
        if _max_score>0.5:
            _peaks.append(_peak_idx)
            _peaks_heights.append(-_peak_height)

    if _peaks == _previous_peaks and len(_peaks)>0 and _n_points in _peaks:
        _peaks_bool=False
    elif len(_peaks) == 0:
        MEH.score[-10+_mean_start_index:10+_mean_start_index] = 0.0

    _n_peaks = len(_peaks)
    _first_peak = _first_peak+min(_peaks)-_left_extension
    _last_peak = _first_peak+max(_peaks)-_left_extension
    _previous_peaks = _peaks

# Pull out time and data information
_score = MEH.score_initial[_first_peak-_left_extension:_last_peak+_right_extension]
_data = MEH.data[_first_peak-_left_extension:_last_peak+_right_extension]
_time = MEH.t[_first_peak-_left_extension:_last_peak+_right_extension]

# Gather useful quantities
_data_scale = np.abs(_data.max()-_data.min())
_range_start_t = [2.0*_default_peak_time,_default_peak_time]
_weights= np.exp(-((_time[:,np.newaxis]-_time[_peaks])**2.0/(2.0*(10.0*_default_peak_time)**2.0))).sum(axis=1)

#_PEAKS = [_p+_mean_start_index-_first_peak for _p in _peaks] # Calculate where the peaks are in the new system

def _mod_template(T,X):
    START_T, SCALE, _PARAMS  = X[0], X[1], X[2:]
    try:
        return SCALE * _template(T-START_T,_PARAMS)
    except RuntimeError:
        print _PARAMS

def _super_model(T,X,_NMODEL):
    OFFSET, GRADIENT, EVENT_PARAMS = X[0], X[1], X[2:].reshape((_NMODEL,_N_PARAMS))
    _model = OFFSET + GRADIENT*(T-T[_n_points]) # The base-line is centred on the first event
    for _model_index in range(_NMODEL):
        _model += _mod_template(T,EVENT_PARAMS[_model_index])
    return _model


def _target(X,_NMODEL=1):
    _model = _super_model(_time,X,_NMODEL)
    return (-(-0.5*(_data-_model)**2.0/NOISE**2.0)*_weights).sum()

# Set up initial location and bounds
_X0 = [MEDIAN,0.0]
_bnds = [(_data.min(), _data.max()),(-1.0,1.0)]
_N_MODEL = len(_peaks)
_N_PARAMS = 2+len(_template_params_names())
for _p, _h in zip(_peaks,_peaks_heights):
    _mean_start_t = _time[_p]-_default_peak_time
    _X0 += [_mean_start_t,-10.0] + _template_params_defaults()
    _bnds += [(_mean_start_t-_range_start_t[0], _mean_start_t+_range_start_t[1]),(1.5*_h, 0.5*_h)] + _template_params_ranges()
_opt = {'gtol':1e-10,'ftol':1e-10,'maxfun':100000}
_res = optimize.minimize(_target,_X0,method='L-BFGS-B', tol=1e-10, bounds=_bnds, options=_opt, args=(_N_MODEL))

0.7334492084602735


In [116]:
_res

      fun: 19.29829069922215
 hess_inv: <8x8 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 3.61666253e-04, -1.03739239e-04,  1.49213975e-04,  1.81188398e-05,
        9.37916411e-05, -6.96331881e-05,  2.21689334e-04, -1.80001898e+00])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 1098
      nit: 86
   status: 0
  success: True
        x: array([-2.54053545e-01, -8.31522519e-02,  6.84060828e+03, -4.60969961e+01,
        1.55307601e-01,  7.13465577e+00,  1.14600508e+00,  9.99000000e-01])

In [117]:
plt.plot(_time,_data)
plt.plot(_time,_score)
plt.plot(_time,_super_model(_time,_res['x'],len(_peaks)))
#plt.plot(_time,_data-_super_model(_time,_res['x'],len(_peaks)))
plt.scatter(_time[_peaks],np.zeros(len(_peaks)),zorder=10,c='orangered')
plt.scatter(_res['x'][2],0,zorder=10,c='orangered')
#plt.scatter(_res['x'][5::4],np.ones(len(_peaks)),zorder=10,c='purple')

In [71]:
_range_start_t

[1.5808690770966969, 0.7904345385483484]

In [ ]:
from scipy import signal
plt.plot(_time,_data)
plt.plot(_time,_score)
tmppeaks,props = signal.find_peaks(-_data,prominence=NOISE*2,height = 5.0,width=2)
truepeaks = np.array([i for i in tmppeaks if _score[i-5:i].max()>0.5])
plt.scatter(_time[tmppeaks],np.zeros(tmppeaks.size),zorder=10,c='purple')
plt.scatter(_time[truepeaks],np.zeros(truepeaks.size),zorder=10,c='orangered')

In [ ]:
plt.plot(_time,_weights)

In [ ]:
int(17200/MEH.dt)

In [ ]:

plt.plot(MEH.t[_idx-_npoints:_idx+_npoints*2],MEH.data[_idx-_npoints:_idx+_npoints*2])
plt.plot(MEH.t[_idx-_npoints:_idx+_npoints*2],MEH.data_residual[_idx-_npoints:_idx+_npoints*2])
plt.plot(MEH.t[_idx-_npoints:_idx+_npoints*2],MEH.score[_idx-_npoints:_idx+_npoints*2])

#plt.scatter(MEH.t[_idx-_npoints:_idx+_npoints*2][highpeaks],np.zeros(highpeaks.size))

In [ ]:
from scipy import signal
plt.plot(MEH.t,MEH.data)
plt.plot(MEH.t,MEH.score)
tmppeaks,props = signal.find_peaks(-MEH.data,prominence=MEH.noise_std*2,height = 5.0,width=2,wlen=20)
truepeaks = np.array([i for i in tmppeaks if MEH.score[i-5:i].max()>3.0])
plt.scatter(MEH.t[truepeaks],np.zeros(truepeaks.size),zorder=10,c='orangered')

In [ ]:
int(16530.0/MEH.dt)

In [ ]:
data = EA.data[-1]
import pandas as pd
def rolling(DATA,WINDOW=50,FUNC='MEDIAN',EDGE_METHOD='SHIFTING'):
    def cleaning(RAW_ROLLING):
        RAW_ROLLING[:WINDOW] = np.roll(RAW_ROLLING,-WINDOW)[:WINDOW]
        RAW_ROLLING[-WINDOW:] = np.roll(RAW_ROLLING,WINDOW)[-WINDOW:]
        return RAW_ROLLING
    
    if FUNC=='MEDIAN':
        raw_rolling = pd.DataFrame(DATA).rolling(window=WINDOW,center=True).median().values.flatten()
    elif FUNC=='MEAN':
        raw_rolling = pd.DataFrame(DATA).rolling(window=WINDOW,center=True).mean().values.flatten()
    elif FUNC=='STD':
        raw_rolling = pd.DataFrame(DATA).rolling(window=WINDOW,center=True).std().values.flatten()
    elif FUNC=='SUM':
        raw_rolling = pd.DataFrame(DATA).rolling(window=WINDOW,center=True).sum().values.flatten()
    elif FUNC=='SMOOTH_MEDIAN':
        raw_rolling_q1 = pd.DataFrame(DATA).rolling(window=WINDOW,center=True).quantile(0.40).values.flatten()
        raw_rolling_q3 = pd.DataFrame(DATA).rolling(window=WINDOW,center=True).quantile(0.60).values.flatten()
        raw_rolling = np.median(np.stack([raw_rolling_q1,raw_rolling_q3]),axis=0)
    elif FUNC=='SUFFICIENT_STATISTICS':
        raw_rolling_q1 = pd.DataFrame(DATA).rolling(window=WINDOW,center=True).quantile(0.25).values.flatten()
        raw_rolling_q2 = pd.DataFrame(DATA).rolling(window=WINDOW,center=True).quantile(0.50).values.flatten()
        raw_rolling_q3 = pd.DataFrame(DATA).rolling(window=WINDOW,center=True).quantile(0.75).values.flatten()
        raw_rolling_std = (raw_rolling_q3-raw_rolling_q1)/1.349
        return cleaning(raw_rolling_q2),cleaning(raw_rolling_std)
        
    return cleaning(raw_rolling)
        

In [ ]:
med, std =rolling(data,FUNC='SUFFICIENT_STATISTICS',WINDOW=MEH.median_window)

In [ ]:
def _triexponential(T,RISE1=1.0,RISE2=2.0,DECAY=3.0,F=0.5):
    NORM = 1.0#1/((DECAY/(RISE+DECAY))*(RISE/(RISE+DECAY))**(RISE/DECAY))
    retarray = np.zeros(T.shape[0])
    postime = T>-10.
    retarray[postime] = NORM*(1.0-F*np.exp(-T[postime]/RISE1)-(1.0-F)*np.exp(-T[postime]/RISE2))*(np.exp(-T[postime]/DECAY))
    return retarray#/retarray.max()

rise1 = 2.0
rise2 = 10.0
decay = 5.0
T = np.linspace(0.,10.,10000)
#frange = np.linspace(-rise1/(rise2-rise1),1.0,100)
frange = np.linspace(0.0,1.0,100)
for f in frange:
    plt.plot(T,_triexponential(T,RISE1=rise1,RISE2=rise2,DECAY=decay,F=f),c=plt.cm.viridis((f-frange.min())/(frange.max()-frange.min())))

In [ ]:
max_x = scipy.optimize.fmin(lambda x: -f(x), 0)

In [129]:
def _triexponential(T,RISE1=1.0,RISE2=2.0,DECAY=3.0,F=0.5):
    NORM = 1.0#1/((DECAY/(RISE+DECAY))*(RISE/(RISE+DECAY))**(RISE/DECAY))
    retarray = np.zeros(T.shape[0])
    retarray = NORM*(1.0-F*np.exp(-T/RISE1)-(1.0-F)*np.exp(-T/RISE2))*(np.exp(-T/DECAY))
    return retarray#/retarray.max()

PARAMS = [4.07268007e-02,6.54606852e+01,9.82617975e-01,9.99000000e-01]
rise1 = PARAMS[0]
rise2 = PARAMS[0]*PARAMS[1]
decay = PARAMS[2]
f = PARAMS[3]
T = np.linspace(0.,100.,100000)
plt.plot(T,_triexponential(T,RISE1=rise1,RISE2=rise2,DECAY=decay,F=f))
#plt.plot(T,utilities._triexponential(T,PARAMS,NORMED=False))
utilities._triexponential_peak(PARAMS)

0.1313133264954047

In [ ]:
# Use minimize_scalar
from math import exp
rise1 = 1.0
rise2 = 2.0
decay = 3.0
f = 0.5
T = np.linspace(0.,10.,10000)
def _triexp_scalar(x):
    return -(1.0-f*exp(-x/rise1)-(1.0-f)*exp(-x/rise2))*exp(-x/decay)
_res = optimize.minimize_scalar(_triexp_scalar)

In [84]:
def _triexp_peak_guess(RISE1=1.0,RISE2=2.0,DECAY=3.0,F=0.5):
    # this one is not great
    #return (F*DECAY*(RISE2-RISE1))/(F*RISE2*(1.0+DECAY/RISE1)+(1.0-F)*RISE1*(1.0+DECAY/RISE2))
    # try biexponential one
    return utilities._biexponential_peak(PARAMS=[RISE2,DECAY])
_triexp_peak_guess(rise1,rise2,decay,f)
_triexp_peak_guess()

1.83258146374831

In [ ]:
from math import exp
rise1 = 1.0
rise2 = 2.0
decay = 3.0
f = 0.5
T = np.linspace(0.,10.,10000)
def _triexp_scalar(x):
    return -(1.0-f*exp(-x/rise1)-(1.0-f)*exp(-x/rise2))*exp(-x/decay)
_res = optimize.minimize_scalar(_triexp_scalar,method='golden',tol=1e-3)

In [ ]:
_res

In [ ]:
%timeit optimize.minimize_scalar(_triexp_scalar,method='brent')

In [ ]:
%timeit optimize.minimize_scalar(_triexp_scalar,method='golden',tol=1e-3)

In [ ]:
%timeit optimize.minimize_scalar(_triexp_scalar,method='bounded',bounds=(0,10))

In [ ]:
from math import exp
def _triexponential_peak(RISE,ETA,DECAY,F):
    RISE1 = RISE
    RISE2 = RISE*ETA
    G0_A = F*(1.0+DECAY/RISE1)
    G0_B = (1.0-F)*(1.0+DECAY/RISE2)
    G1_A = -G0_A/RISE1
    G1_B = -G0_B/RISE2
    G2_A = -G1_A/RISE1
    G2_B = -G1_B/RISE2
    def _G0(x):
        return -1.0+G0_A*exp(-x/RISE1)+G0_B*exp(-x/RISE2)
    def _G1(x):
        return G1_A*exp(-x/RISE1)+G1_B*exp(-x/RISE2)
    def _G2(x):
        return G2_A*exp(-x/RISE1)+G2_B*exp(-x/RISE2)

    root = optimize.newton(_G0, 1.0, fprime=_G1,fprime2=_G2)
    return root

def _triexponential(T,RISE=1.0,ETA=1.0,DECAY=1.0,F=0.5,NORMED=True):
    RISE1 = RISE
    RISE2 = RISE*ETA
    retarray = np.zeros(T.shape[0])
    retarray = (1.0-F*np.exp(-T/RISE1)-(1.0-F)*np.exp(-T/RISE2))*(np.exp(-T/DECAY))
    if NORMED == True:
        PEAK = _triexponential_peak(RISE,ETA,DECAY,F)
        NORM = _triexponential(np.array([PEAK]),RISE,ETA,DECAY,F,NORMED=False)[0]
        retarray /= NORM
    return retarray

def _triexponential_area(RISE=1.0,ETA=1.0,DECAY=1.0,F=0.5,NORMED=True):
    RISE1 = RISE
    RISE2 = RISE*ETA
    AREA = ((DECAY+(1.0-F)*RISE1+F*RISE2)/((1.0+RISE1/DECAY)*(1.0+RISE2/DECAY)))
    if NORMED == True:
        PEAK = _triexponential_peak(RISE,ETA,DECAY,F)
        NORM = _triexponential(np.array([PEAK]),RISE,ETA,DECAY,F,NORMED=False)[0]
        AREA /= NORM
    return AREA

In [ ]:
rise = 1.0
eta = 2.0
decay = 5.0
f = 0.5
T = np.linspace(0.,10.,10000)
#varrange = np.logspace(0.0,5,100)
varrange = np.linspace(0.,1.0,100)

for v in varrange:
    plt.plot(T,_triexponential(T,RISE=rise,ETA=eta,DECAY=decay,F=v),c=plt.cm.viridis((v)/(1.0)))
plt.plot(T,_triexponential(T,RISE=rise,ETA=eta,DECAY=decay,F=0.5),c='r')

In [ ]:
rise = 1.0
eta = 2.0
decay = 5.0
f = 0.5
T = np.linspace(0.,10.,10000)
varrange = np.logspace(0.0,10,100)

for v in varrange:
    plt.plot(T,_triexponential(T,RISE=rise,ETA=v,DECAY=decay,F=f),c=plt.cm.viridis(np.log10(v)/(5.0)))
plt.plot(T,_triexponential(T,RISE=rise,ETA=eta,DECAY=decay,F=0.5),c='r')

In [ ]:
np.log10(varrange)

In [ ]:
def integrand(t):
    return _triexponential(np.array([t]),RISE=1.0,ETA=2.0,DECAY=5.0,F=0.8,NORMED=True)[0]
integrate.quad(integrand,0,np.inf)

In [ ]:
_triexponential_area(RISE=1.0,ETA=2.0,DECAY=5.0,F=0.8)

In [ ]:
%timeit _triexponential_peak(RISE1=1.0,RISE2=2.0,DECAY=5.0,F=0.5)

In [ ]:
%timeit optimize.newton(_G0, 0.0)

In [ ]:
import numpy as np
from scipy import sparse
from scipy.sparse.linalg import spsolve

def baseline_als(y, lam, p, niter=10):
    L = len(y)
    D = sparse.diags([1,-2,1],[0,-1,-2], shape=(L,L-2))
    w = np.ones(L)
    for i in range(niter):
        W = sparse.spdiags(w, 0, L, L)
        Z = W + lam * D.dot(D.transpose())
        z = spsolve(Z, w*y)
        w = p * (y > z) + (1-p) * (y < z)
    return z

In [ ]:
plt.plot(MEH.t,EA.data[-1])
plt.plot(MEH.t,baseline_als(EA.data[-1],10000.,0.001))

In [ ]:
def highpass_filter(y, sr):
    filter_stop_freq = 10  # Hz
    filter_pass_freq = 20  # Hz
    filter_order = 1001

    # High-pass filter
    nyquist_rate = sr / 2.
    desired = (0, 0, 1, 1)
    bands = (0, filter_stop_freq, filter_pass_freq, nyquist_rate)
    filter_coefs = signal.firls(filter_order, bands, desired, nyq=nyquist_rate)

    # Apply high-pass filter
    filtered_audio = signal.filtfilt(filter_coefs, [1], y)
    return filtered_audio

In [ ]:
plt.plot(MEH.t,EA.data[-1])
plt.plot(MEH.t,highpass_filter(EA.data[-1],1e3/MEH.dt))

In [ ]:
MEH.dt